In [ ]:
from pathlib import Path
import pandas as pd
from pytorch_mnl.core import *

In [ ]:
PATH = Path('data')

In [ ]:
df = pd.read_csv(PATH/'swissmetro.dat', '\t')

In [ ]:
df.head()

,GROUP,SURVEY,SP,ID,PURPOSE,FIRST,TICKET,WHO,LUGGAGE,AGE,...,TRAIN_TT,TRAIN_CO,TRAIN_HE,SM_TT,SM_CO,SM_HE,SM_SEATS,CAR_TT,CAR_CO,CHOICE
0,2,0,1,1,1,0,1,1,0,3,...,112,48,120,63,52,20,0,117,65,2
1,2,0,1,1,1,0,1,1,0,3,...,103,48,30,60,49,10,0,117,84,2
2,2,0,1,1,1,0,1,1,0,3,...,130,48,60,67,58,30,0,117,52,2
3,2,0,1,1,1,0,1,1,0,3,...,103,40,30,63,52,20,0,72,52,2
4,2,0,1,1,1,0,1,1,0,3,...,130,36,60,63,42,20,0,90,84,2


Remove some observations


In [ ]:
exclude = df.query('(PURPOSE != 1 and PURPOSE != 3) or CHOICE == 0')

In [ ]:
df = df.drop(exclude.index)
len(df)

6768

In [ ]:
df.columns.to_list()

['GROUP',
 'SURVEY',
 'SP',
 'ID',
 'PURPOSE',
 'FIRST',
 'TICKET',
 'WHO',
 'LUGGAGE',
 'AGE',
 'MALE',
 'INCOME',
 'GA',
 'ORIGIN',
 'DEST',
 'TRAIN_AV',
 'CAR_AV',
 'SM_AV',
 'TRAIN_TT',
 'TRAIN_CO',
 'TRAIN_HE',
 'SM_TT',
 'SM_CO',
 'SM_HE',
 'SM_SEATS',
 'CAR_TT',
 'CAR_CO',
 'CHOICE',
 'SM_COST',
 'TRAIN_COST',
 'CAR_AV_SP',
 'TRAIN_AV_SP',
 'TRAIN_TT_SCALED',
 'TRAIN_COST_SCALED',
 'SM_TT_SCALED',
 'SM_COST_SCALED',
 'CAR_TT_SCALED',
 'CAR_CO_SCALED']

Model Params

In [ ]:
# ASC_CAR = Beta('ASC_CAR', 0, None, None, 0)
# ASC_TRAIN = Beta('ASC_TRAIN', 0, None, None, 0)
# ASC_SM = Beta('ASC_SM', 0, None, None, 1)
# B_TIME = Beta('B_TIME', 0, None, None, 0)
# B_COST = Beta('B_COST', 0, None, None, 0)

Definition of new variables

In [ ]:
df = df.assign(SM_COST = df['SM_CO'] * (df['GA'] == 0))
df = df.assign(TRAIN_COST = df['TRAIN_CO'] * (df['GA'] == 0))
df = df.assign(CAR_AV_SP = df['CAR_AV'] * (df['SP'] != 0))
df = df.assign(TRAIN_AV_SP = df['TRAIN_AV'] * (df['SP'] != 0))
df = df.assign(TRAIN_TT_SCALED = df['TRAIN_TT'] / 100)
df = df.assign(TRAIN_COST_SCALED = df['TRAIN_COST'] / 100)
df = df.assign(SM_TT_SCALED = df['SM_TT'] / 100)
df = df.assign(SM_COST_SCALED = df['SM_COST'] / 100)
df = df.assign(CAR_TT_SCALED = df['CAR_TT'] / 100)
df = df.assign(CAR_CO_SCALED = df['CAR_CO'] / 100)

In [ ]:
df.columns.to_list()

['GROUP',
 'SURVEY',
 'SP',
 'ID',
 'PURPOSE',
 'FIRST',
 'TICKET',
 'WHO',
 'LUGGAGE',
 'AGE',
 'MALE',
 'INCOME',
 'GA',
 'ORIGIN',
 'DEST',
 'TRAIN_AV',
 'CAR_AV',
 'SM_AV',
 'TRAIN_TT',
 'TRAIN_CO',
 'TRAIN_HE',
 'SM_TT',
 'SM_CO',
 'SM_HE',
 'SM_SEATS',
 'CAR_TT',
 'CAR_CO',
 'CHOICE',
 'SM_COST',
 'TRAIN_COST',
 'CAR_AV_SP',
 'TRAIN_AV_SP',
 'TRAIN_TT_SCALED',
 'TRAIN_COST_SCALED',
 'SM_TT_SCALED',
 'SM_COST_SCALED',
 'CAR_TT_SCALED',
 'CAR_CO_SCALED']

## Utility Func
> you have to define your model as a nn.Module

In [ ]:
# V1 = ASC_TRAIN + \
#      B_TIME * TRAIN_TT_SCALED + \
#      B_COST * TRAIN_COST_SCALED
# V2 = ASC_SM + \
#      B_TIME * SM_TT_SCALED + \
#      B_COST * SM_COST_SCALED
# V3 = ASC_CAR + \
#      B_TIME * CAR_TT_SCALED + \
#      B_COST * CAR_CO_SCALED

$$V = a + b\cdot x $$

In [ ]:
import torch
import torch.nn as nn

class Model(nn.Module):
    def __init__(self, n_choices=3, n_params=2):
        super().__init__()
        self.b = nn.Parameter(torch.randn(n_params,1))
        self.a = nn.Parameter(torch.randn(n_choices, 1))
        
    def forward(self, x):
        return (self.a + x @ self.b).squeeze()

In [ ]:
model = Model()

In [ ]:
model(torch.rand(8, 3, 2)).shape

torch.Size([8, 3])

In [ ]:
x_cols =  ['TRAIN_TT_SCALED',
           'TRAIN_COST_SCALED',
           'SM_TT_SCALED',
           'SM_COST_SCALED',
           'CAR_TT_SCALED',
           'CAR_CO_SCALED']

In [ ]:
torch.arange(0,6).reshape(3,2)

tensor([[0, 1],
        [2, 3],
        [4, 5]])

In [ ]:
X, y = prepare_data(df, x_cols=x_cols, target_col='CHOICE')

X = X.reshape(-1,3,2)

In [ ]:
dls = DataLoaders.from_Xy(X, y, pct=0.2, batch_size=8)

In [ ]:
x,y = dls.one_batch()

In [ ]:
model(x)

tensor([[ 0.5581,  0.9800, -0.0425],
        [-0.0487,  0.4095,  0.4071],
        [ 0.3338,  1.0630,  0.0887],
        [-0.0885,  0.8173, -0.0624],
        [-0.0015,  0.6939,  0.3146],
        [ 0.1010,  1.1376,  0.5224],
        [ 0.5323,  1.0341,  0.1136],
        [ 0.7406,  1.3480, -0.0027]], grad_fn=<SqueezeBackward0>)

In [ ]:
loss_func = nn.CrossEntropyLoss()
loss_func(model(x), y)

tensor(1.4041, grad_fn=<NllLossBackward>)

In [ ]:
learn = Learner(dls, model, loss_func)

In [ ]:
learn.fit(5, lr=0.1)

epoch =   0, train_loss = 611.240, val_loss = 75.780, accuracy = 0.60
epoch =   1, train_loss = 611.240, val_loss = 75.780, accuracy = 0.60
epoch =   2, train_loss = 611.240, val_loss = 75.780, accuracy = 0.60
epoch =   3, train_loss = 611.240, val_loss = 75.780, accuracy = 0.60
epoch =   4, train_loss = 611.240, val_loss = 75.780, accuracy = 0.60


In [ ]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.2348],
         [-0.7490]], requires_grad=True),
 Parameter containing:
 tensor([[ 0.6674],
         [-0.6521],
         [ 0.0133]], requires_grad=True)]